# Importowanie bibliotek
W projekcie wykorzystano następujące biblioteki:
- TensorFlow i Keras: Do budowy i trenowania modelu głębokiego uczenia.
- NumPy i Pandas: Do manipulacji danymi i ich wstępnego przetwarzania.
- Scikit-learn: Do skalowania danych oraz podziału na zbiory treningowe i testowe.
- Keras Tuner: Do optymalizacji hiperparametrów modelu.
- Callbacks w Keras: EarlyStopping i ReduceLROnPlateau zapewniają zatrzymanie treningu w odpowiednim momencie i dostosowywanie tempa uczenia.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner.tuners import BayesianOptimization
from tensorflow.keras.layers import Input

# Przygotowanie danych
- Wczytanie danych: Dane zostały zaimportowane z pliku CSV.
- Wybór cech: Wybrano kolumny numeryczne: ['Size(L)', 'OG', 'FG', 'IBU', 'ABV', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency'].
- Uzupełnienie braków: Wartości brakujące w danych zastąpiono średnią z kolumn.
- Normalizacja: Dane zostały znormalizowane przy użyciu StandardScaler, co pozwala na lepszą konwergencję modelu.
- Podział danych: Dane podzielono na zbiór treningowy (80%) i testowy (20%) z losowym seedem 42.

In [2]:
dane = pd.read_csv('../data.csv')

kolumny_numeryczne = ['Size(L)', 'OG', 'FG', 'IBU', 'ABV', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = dane[kolumny_numeryczne].copy()

X = X.fillna(X.mean())

y = dane['Color'].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Struktura modelu
Model został zbudowany w formie sekwencyjnej.

Warstwy:
- Warstwa wejściowa: 288 neuronów z aktywacją ReLU.
- Cztery warstwy ukryte:
  - Każda warstwa ma regulowaną liczbę neuronów (od 32 do 512, w krokach po 32) oraz Dropout (od 0.0 do 0.5).
  - Każda warstwa jest wsparta Batch Normalization, stabilizującą proces uczenia.
- Warstwa wyjściowa: Jeden neuron do przewidywania koloru piwa (regresja).

Regularyzacja: Dropout zapobiega przeuczeniu, a Batch Normalization przyspiesza i stabilizuje proces uczenia.

Kompilacja modelu
- Optymalizator: Adam z dynamiczną szybkością uczenia (od 1e-4 do 1e-2, logarytmiczny zakres).
- Funkcja straty: Mean Squared Error (MSE).
- Metryka: Mean Absolute Error (MAE).

In [3]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(
        units=hp.Int('units_input', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_1', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_2', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(
        units=hp.Int('units_3', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(1))
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='mean_squared_error',
        metrics=['mean_absolute_error']
    )
    
    return model


# Proces optymalizacji hiperparametrów
Keras Tuner:
- Rodzaj tunera: BayesianOptimization.
- Cel: Minimalizacja val_mean_absolute_error.
- Liczba prób: 20.
- Callbacki: EarlyStopping (patience=5) i TensorBoard do wizualizacji.

Wynik:
- Najlepsze val_mean_absolute_error: 7.354.
- Łączny czas optymalizacji: ~21 minut.

In [4]:
tuner = BayesianOptimization(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=20,
    directory='logs/fit',
    project_name='regresja_koloru_piwa')

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='logs/fit/regresja_koloru_piwa')

tuner.search(X_train, y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback],
             batch_size=32)

Trial 20 Complete [00h 00m 45s]
val_mean_absolute_error: 7.5820794105529785

Best val_mean_absolute_error So Far: 7.354419708251953
Total elapsed time: 00h 21m 06s


In [5]:
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 384)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │       110,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 416)            │       120,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 416)            │         1,664 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 288)            │       120,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 360,833 (1.38 MB)

 Trainable params: 358,081 (1.37 MB)

 Non-trainable params: 2,752 (10.75 KB)

# Wyniki najlepszego modelu

Proces treningu:
 - Liczba epok: 20.
 - Batch size: 32.

Najlepszy wynik na walidacji:
 - Final Validation MAE: 7.66.

In [6]:
history = best_model.fit(X_train, y_train, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

best_model.summary()

final_val_mae = history.history['val_mean_absolute_error'][-1]
print(f"Final Validation MAE: {final_val_mae:.2f}")

Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 113.9864 - mean_absolute_error: 7.9553 - val_loss: 112.0139 - val_mean_absolute_error: 8.1053
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 114.1593 - mean_absolute_error: 7.9720 - val_loss: 113.6976 - val_mean_absolute_error: 8.2092
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 114.0531 - mean_absolute_error: 7.9811 - val_loss: 111.8362 - val_mean_absolute_error: 7.7278
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 114.1284 - mean_absolute_error: 7.9244 - val_loss: 110.8494 - val_mean_absolute_error: 7.9674
Epoch 5/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 114.9939 - mean_absolute_error: 7.9336 - val_loss: 112.8694 - val_mean_absolute_error: 7.5307
Epoch 6/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 114.8706 - mean_absolute_error: 7.9531 - val_loss: 110.9353 - val_mean_absolute_error: 7.6213
Epoch 7/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 11

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 384)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │       110,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 416)            │       120,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 416)            │         1,664 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 288)            │       120,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,076,997 (4.11 MB)

 Trainable params: 358,081 (1.37 MB)

 Non-trainable params: 2,752 (10.75 KB)

 Optimizer params: 716,164 (2.73 MB)

Final Validation MAE: 7.66


# Zapisywanie Modelu

In [7]:
best_model.save('../models/tuned_model_regresji_koloru_piwa.h5')

wczytany_model = keras.models.load_model('../models/tuned_model_regresji_koloru_piwa.h5')

wczytany_model.summary()

wczytany_test_loss, wczytany_test_mae = wczytany_model.evaluate(X_test, y_test)

print(f'Wczytana Testowa Strata: {wczytany_test_loss}')
print(f'Wczytany Testowy Średni Błąd Bezwzględny: {wczytany_test_mae}')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 384)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 384)            │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 288)            │       110,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 416)            │       120,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 416)            │         1,664 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 288)            │       120,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 288)            │         1,152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 360,835 (1.38 MB)

 Trainable params: 358,081 (1.37 MB)

 Non-trainable params: 2,752 (10.75 KB)

 Optimizer params: 2 (12.00 B)

462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - loss: 117.6217 - mean_absolute_error: 8.0027
Wczytana Testowa Strata: 116.81678009033203
Wczytany Testowy Średni Błąd Bezwzględny: 7.99605655670166
